<h1><center>CS457 Lab 1 <br />Introduction to TCP Sockets</center></h1>

### Introduction

This first CS457 lab is a "gentle" introduction to network socket programming.  For the time being we will avoid handling errors and other situations such as dealing with network byte order or asynchronous multi-connection threads.  We'll get to those issues in the next lab.
Right now we just want study some simple examples to give you basic information on how sockets work. 

#### Lab 1 Overview

In this lab you will simply execute  python code to study examples of simple socket programs; no new code needs to be written by you. Executing a cell in jupyter requires you to be in the cell and pushing SHIFT RETURN. 

In this lab you will learn how to use TCP sockets to

1. Connect to a well known web site, google.com, and read the contents of a web page.  
2. Run a simple "hello world" server.
3. Run very simple "echo server" and "echo client" programs to illustrate the differences in structure between a TCP server and a TCP client.

#### Acknowledgements and Citations

This lab has original material as well as material sourced and modified from various sites such as Nathan Jenning's [Socket Programming in Python](https://realpython.com/python-sockets/), the [GeeksForGeeks Socket programming guide](https://www.geeksforgeeks.org/socket-programming-python/), and lab exercises from the Kurose and Ross Networking textbook.

----

### Part 1:  Introduction to Sockets and Ports

Socket programming is a way of connecting two nodes on a network to communicate with each other. In streaming mode (TCP), one socket(node) listens on a particular port, while another socket reaches out to the other to form a connection.  Data can then be exchanged in a client/server relationship.

Ports are a method for the operating system to connect the socket to the appropriate application.  

<img src="http://www.cs.colostate.edu/~cs457/images/jupyter/socket_port.png" alt="socket diagram" />

Ports are simply numbers from 0 to 65535.  The first 1023 ports are reserved for specific applications, and are called "well known ports". As illustrated below, Port 80, for example, is used for web servers.  Port 53 is used for DNS.  Here is a list of [well known ports](https://en.wikipedia.org/wiki/List_of_TCP_and_UDP_port_numbers). The ports numbered from 1024 and above are called *"ephemeral ports"* and can be used by any application.

<img src="http://www.cs.colostate.edu/~cs457/images/jupyter/multipleSockets.jpg" width="350px" alt="multiple sockets" />

#### 1A) Socket Creation and Initialization

Socket programming in python is started by importing the socket library and making a simple socket. Execute the following cell (SHIFT-RETURN):

In [1]:
import socket
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
print (s)

<socket.socket fd=404, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0>


Here we made a socket instance and passed it two parameters. The first parameter is `AF_INET` and the second one is `SOCK_STREAM`.  `AF_INET` refers to the IPv4 address family (later we will use IPv6). The `SOCK_STREAM` parameter means to use a socket with *connection-oriented* TCP protocol. In a future lab we will use the SOCK_DATAGRAM parameter to do UDP messaging.  

Notice that when we printed the socket object, it indicated an `fd` (*file descriptor*).  Each socket is assigned a unique `fd`. There are many similarities between an `fd` for real files and an `fd` for sockets.  Both have common operations such as `open, read, write`, and `close`.  Of course, there are also fundamental differences between files and sockets -- for example, files don't have `listen` and sockets don't have `seek`.  We'll get to the socket API calls in the next section. 

In any event, we can now connect to a server using this socket.  But note that if any error occurs during the creation of a socket then a socket.error is thrown.  Furthermore, we can only connect to a server by knowing it’s IP address or by specifying it's fully qualified domain name.  In this lab we will use IPv4 addresses only.

   #### 1B) Finding the IP address of a server: 

You can find the IP address of a server by using the unix `ping` command.  The `ping` command sends packets of data to a specific IP address on a network, and then lets you know how long it took to transmit that data and get a response. It’s a handy tool that you can use to quickly test various points of your network.  In the example below we send 3 pings to www.google.com  (the "!" symbol causes jupyter to execute a unix shell command).  Execute the ping command and examine the output.

In [2]:
! ping www.google.com -c 3

Access denied. Option -c requires administrative privileges.


You can also find the IP address using the python socket's `gethostbyname` method:

In [3]:
import socket 
ip = socket.gethostbyname('www.google.com')
print (ip)

172.217.11.228


#### 1C) Connecting to a server:  An example script to connect to google

The code block below executes the following flow:

- First of all we make a socket.

- Then we resolve google’s IP address and then we connect to google using port 80, the reserved port number used by web servers that use the HTTP protocol.

- Once connected we use the **`send`** API call send a request to the server.  The message we send is an `HTTP GET` request for a web page.  Don't worry about the format of the `GET` request for now.

- Finally we implement a block of code to receive the first block of data from the server, followed by a loop to receive the rest of the data from the server.  Why a loop?  Because the data being returned may be bigger than fits into one packet.  TCP is a STREAMING connection; we may get a lot of data back in multiple packets.  These packets are buffered inside the operating systems network stack.   We use the socket **`recv`** API to read the data and to tell the socket to give the python application no more than 10,000 bytes at each invocation.  We check the length of the data returned.  Once it hits 0, we are done. 

- Finally, we close the connection.

See!  Easy-peasy!  But as useful as sockets are, you should be aware that there are more advanced python libraries for fetching and parsing web pages.  These include ['urllib'](https://docs.python.org/3/library/urllib.html), ['httplib'](https://docs.python.org/3/library/http.client.html) and the very useful['requests'](https://realpython.com/python-requests/) library, among others.  The Python Library called [BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) is an awesome set of tools to parse HTTP and XML files.  We will get to these in future labs.  For now we are learning about basic socket operations.

Execute the code block and examine the output.  You should see multiple blocks of data returned, each separated by a blank line.  


In [1]:
# An example script to connect to Google using socket programming in Python 

import socket # for socket 
import sys 

try: 
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM) 
    print ("Socket successfully created")
except socket.error as err: 
    print ("socket creation failed with error %s") %(err) 

# default port for socket 
port = 80

try: 
    hostname = "www.google.com"
    host_ip = socket.gethostbyname(hostname) 
except socket.gaierror: 

    # this means could not resolve the host 
    print ("there was an error resolving the host")
    sys.exit() 

# connecting to the server 
s.connect((host_ip, port)) 

print ("the socket has successfully connected to", hostname,  "on address", host_ip, ", port", port)

#  now get a web page of HTML data.  
#  The response will print as raw ASCII HTML text; it will not be rendered as done in a web browser.
#  the "b" character in front of the string means "byte data", it will not be convered to a python string.

request = b"GET / HTTP/1.1\r\nHost: www.google.com\r\n\r\n"
s.send(request)
result = s.recv(10000)
while (len(result) > 0):
    print("\n", result)
    result = s.recv(10000) 
s.close()

Socket successfully created
the socket has successfully connected to www.google.com on address 172.217.2.4 , port 80

 b'HTTP/1.1 200 OK\r\nDate: Thu, 29 Aug 2019 19:14:42 GMT\r\nExpires: -1\r\nCache-Control: private, max-age=0\r\nContent-Type: text/html; charset=ISO-8859-1\r\nP3P: CP="This is not a P3P policy! See g.co/p3phelp for more info."\r\nServer: gws\r\nX-XSS-Protection: 0\r\nX-Frame-Options: SAMEORIGIN\r\nSet-Cookie: 1P_JAR=2019-08-29-19; expires=Sat, 28-Sep-2019 19:14:42 GMT; path=/; domain=.google.com; SameSite=none\r\nSet-Cookie: NID=188=aRZIrBW_gGtHxxmgXar4JFGjo815l0lWZlpUxwBpwdpkWU-12hWi9NlvWTwxyiU3i6TCHrbNDeeGdhb1O-JHRTn7d6NecuZICEuvaPd91gObDJVCnQNIlpy_rwnVOUTU6gH9SiqcuoaPFh46Vgs7IE-Zdy5VNxkmWkxXrYfmj3s; expires=Fri, 28-Feb-2020 19:14:42 GMT; path=/; domain=.google.com; HttpOnly\r\nAccept-Ranges: none\r\nVary: Accept-Encoding\r\nTransfer-Encoding: chunked\r\n\r\n5fda\r\n<!doctype html><html itemscope="" itemtype="http://schema.org/WebPage" lang="en"><head><meta content="


 b'n"><div class="ds" style="height:32px;margin:4px 0"><input style="color:#000;margin:0;padding:5px 8px 0 6px;vertical-align:top" autocomplete="off" class="lst" value="" title="Google Search" maxlength="2048" name="q" size="57"></div><br style="line-height:0"><span class="ds"><span class="lsbb"><input class="lsb" value="Google Search" name="btnG" type="submit"></span></span><span class="ds"><span class="lsbb"><input class="lsb" value="I\'m Feeling Lucky" name="btnI" onclick="/*gws_metadata_pragma__js_inline_handler*/ if(this.form.q.value){ this.checked=1; if (this.form.iflsig)this.form.iflsig.disabled=false; } else top.location=\'/doodles/\'" type="submit"></span></span></td><td class="fl sblc" align="left" nowrap="" width="25%"><a href="/advanced_search?hl=en&amp;authuser=0">Advanced search</a><a href="/language_tools?hl=en&amp;authuser=0">Language tools</a></td></tr></table><input id="gbv" name="gbv" type="hidden" value="1"><script nonce="DeyMoqrtrjJ6sIyt2LbQDQ==">(function(){var a

#### 1D) Build and run a very simple server application

In this section you will build a server application that does nothing more than send a greeting message to a client.

You will notice the code below is similar to, but a bit more complicated, than the code for a client application.  A server has a `bind()` method which binds it to a specific ip and port so that it can listen to incoming requests on that ip and port.  A server has a `listen()` method which puts the server into listen mode. This allows the server to listen to incoming connections. Lastly, a server has an `accept()` and `close()` method. The accept method initiates a connection with the client and the close method closes the connection with the client.  We will go into more detail on these API calls in section 2 of this lab.  For now, don't worry about understanding everything, just get the general idea.

Here is the program flow:

* First of all we import socket which is necessary.
* Then we make a socket object and reserve a port on our host system.
* After that we `bind` our server to the specified port. Passing an empty string means that the server can listen to incoming connections from other computers as well. If we would have passed 127.0.0.1 then it would have listened to only those calls made within the local computer.
* After that we put the server into listen mode. The parameter "5" here means that 5 connections are kept waiting if the server is busy and if a 6th socket trys to connect then the connection is refused.
* At last we make a while loop and start to accept all incoming connections and close those connections after a thank you message to all connected sockets.

Some special notes:

* This program runs forever.  You may have to use the **`kernel`** menu item in jupyter to **restart and clear output** to kill this program.

* If you get an error that states `this address already in use`, either wait for the old socket to timeout (several minutes) or change the port number.

Now execute the code block below and go to the following section of text for your next set of instructions.

In [ ]:
# a very simple web server that simply greets the client and disconnects

# first of all import the socket library 
import socket 

# next create a socket object 
s = socket.socket()	
print ("Socket successfully created")

# reserve a port on your computer in our 
# case it is 12345 but it can be anything >= 1024 and <=65535
port = 12345

# Next bind to the port 
# we have not typed any ip in the ip field 
# instead we have inputted an empty string 
# this makes the server listen to requests 
# coming from other computers on the network 
s.bind(('', port)) 
print ("socket bound to", port, "on host IP address")

# put the socket into listening mode 
s.listen(5)
print ("socket is listening")

# a forever loop until we interrupt it or 
# an error occurs 

while True: 

    # Establish connection with client. 
    c, addr = s.accept() 
    print ("Got connection from", addr) 

    # send a thank you message to the client. 
    c.send(b'\nYou are such a nice client.\nThank you for connecting, however I am going to disconnect now!  Bye!\n\n') 

    # Close the connection with the client 
    c.close() 
    
s.close()


Socket successfully created
socket bound to 12345 on host IP address
socket is listening


After executing the code block above, you should see output showing that the server is running and listening on port 12345.  Now let's communicate with your mini-server!

Use a separate linux terminal window for this next step. Jupyter doesn't let you run telnet from within a jupyter cell. 

Make sure you have telnet installed on your laptop.  You may have to manually install it; for example, it does not come in the default MACOS operating system.  Once you determine it is available go to your terminal window and execute the following command:

  **`telnet localhost 12345`**  
   
   or 
   
   **`telnet 127.0.0.1 12345`**   
   
   (127.0.0.1 is the IP loopback address on your host machine, also known as "`localhost`".
   
Repeat the telnet command several times.  You should get the server's message text each time you execute the command.  Notice that each time you connect to the server the output block in the code cell above adds an output line stating that a connection has been made.  Each connection is from some random port number chosen by the telnet command.

Now you have to kill the server program.  Use the jupyter menu bar, select **`kernel`** and then select **`restart & clear all output`**.  Give it a minute or so to shut down and restart.  You will know you are ready to continue when the output messages from python cells are cleared out.

--- 

### Part 2:  A deeper look --  The Python Socket API



#### Socket API Overview
Python’s socket module provides an interface to the Berkeley sockets API. This is the module that we’ll use and discuss in this tutorial.

The primary socket API functions and methods in this module are:

```
* socket()
* bind()
* listen()
* accept()
* connect()
* connect_ex()
* send()
* sendall()
* recv()
* gethostbyname()
* close()
```

Python provides a convenient and consistent API that maps directly to these system calls, their C counterparts. We’ll look at how these are used together in the next section.

As part of its standard library, Python also has classes that make using these low-level socket functions easier. Although it’s not covered in this tutorial, see the socketserver module, a framework for network servers. There are also many modules available that implement higher-level Internet protocols like HTTP and SMTP. For an overview, see Internet Protocols and Support.

Using the diagram below, let’s look at the sequence of socket API calls and data flow for TCP:


#### TCP Sockets

As you’ll see shortly, we’ll create a socket object using socket.socket() and specify the socket type as socket.SOCK_STREAM. When you do that, the default protocol that’s used is the Transmission Control Protocol (TCP). This is a good default and probably what you want.

<img src="http://www.cs.colostate.edu/~cs457/images/jupyter/socketToSocket.jpg" width="500px" alt="socket to socket" />

Why should you use TCP? The Transmission Control Protocol (TCP):

* Is reliable: packets dropped in the network are detected and retransmitted by the sender.
* Has in-order data delivery: data is read by your application in the order it was written by the sender.

In contrast, User Datagram Protocol (UDP) sockets created with socket.SOCK_DGRAM aren’t reliable, and data read by the receiver can be out-of-order from the sender’s writes.

Why is this important? Networks are a best-effort delivery system. There’s no guarantee that your data will reach its destination or that you’ll receive what’s been sent to you.

Network devices (for example, routers and switches), have finite bandwidth available and their own inherent system limitations. They have CPUs, memory, buses, and interface packet buffers, just like our clients and servers. TCP relieves you from having to worry about packet loss, data arriving out-of-order, and many other things that invariably happen when you’re communicating across a network.

In the diagram below, let’s look at the sequence of socket API calls and data flow for TCP:

<img src="http://www.cs.colostate.edu/~cs457/images/jupyter/sockets-tcp-flow.jpg" width="500px" alt="TCP socket flow" />

The left-hand column represents the server. On the right-hand side is the client.

Starting in the top left-hand column, note the API calls the server makes to setup a “listening” socket:

```
* socket()
* bind()
* listen()
* accept()
```

A listening socket does just what it sounds like. It listens for connections from clients. When a client connects, the server calls accept() to accept, or complete, the connection.

The client calls connect() to establish a connection to the server and initiate the three-way handshake. The handshake step is important since it ensures that each side of the connection is reachable in the network, in other words that the client can reach the server and vice-versa. It may be that only one host, client or server, can reach the other.

In the middle is the round-trip section, where data is exchanged between the client and server using calls to send() and recv().

At the bottom, the client and server close() their respective sockets.

#### 2A) Create the Echo Server

Now that you’ve seen an overview of the socket API and how the client and server communicate, let’s create our first client and server. We’ll begin with a simple implementation. The server will simply echo whatever it receives back to the client.

Here is the code for the server, `echo-server.py`.  You will notice it uses a slightly more sophisticated python structures than the previous examples.  We'll explain the code in the next section.

For now, just SHIFT-RETURN in the cell below to write the file to your current working directory where this jupyter notebook is located.  You will run the server soon after you examine the client program.


In [ ]:
%%writefile cs457_echoServer.py

# TCP echo server; performs one echo and then closes and halts

import socket

HOST = '127.0.0.1'  # Standard loopback interface address (localhost)
PORT = 65432        # Port to listen on (non-privileged ports are > 1023)

#use "with" so we don't have to explicitly close the socket
with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:  
    s.bind((HOST, PORT))
    s.listen()
    conn, addr = s.accept()
    with conn:
        print('Connected by', addr)
        while True:
            data = conn.recv(1024)
            if not data:
                break
            conn.sendall(data)

Let’s walk through each API call and see what’s happening.

* 'socket.socket()' creates a socket object that supports the context manager type, so you can use it in a `with` statement. There’s no need to call s.close() because the `with` statement will close the connection automatically upon its exit.

* 'bind()' is used to associate the socket with a specific network interface and port number.  The values passed to bind() depend on the address family of the socket. In this example, we’re using socket.AF_INET (IPv4). So it expects a 2-tuple: (host, port).  host can be a hostname, IP address, or empty string. If an IP address is used, host should be an IPv4-formatted address string. The IP address 127.0.0.1 is the standard IPv4 address for the loopback interface, so only processes on the host will be able to connect to the server. If you pass an empty string, the server will accept connections on all available IPv4 interfaces.

* 'listen()' enables a server to accept() connections. It makes it a “listening” socket.  `listen()` has a backlog parameter. It specifies the number of unaccepted connections that the system will allow before refusing new connections. Starting in Python 3.5, it’s optional. If not specified, a default backlog value is chosen.  If your server receives a lot of connection requests simultaneously, increasing the backlog value may help by setting the maximum length of the queue for pending connections. The maximum value is system dependent. 

* `accept()` blocks and waits for an incoming connection. When a client connects, it returns a new socket object representing the connection and a tuple holding the address of the client. The tuple will contain (host, port) for IPv4 connections or (host, port, flowinfo, scopeid) for IPv6.   **One thing that’s imperative to understand is that we now have a new socket object from accept().** This is important since the `conn` socket is the socket that you’ll use to communicate with the client. It is distinct from the *listening socket* that the server is using to accept new connections.

* After getting the client socket object conn from `accept()`, an infinite `while` loop is used to loop over blocking calls to `conn.recv()`. This reads whatever data the client sends and echoes it back using `conn.sendall()`.  If `conn.recv()` returns an empty bytes object, b'', then the client closed the connection and the loop is terminated. The `with` statement is used with `conn` to automatically close the socket at the end of the block.

#### 2B) Create the Echo Client 

This echo client code follows the logic illustrated on the right hand side of the client-server block diagram pictured earlier in this lab.

In comparison to the server, the client is pretty simple. It creates a socket object, connects to the server and calls s.sendall() to send its message.  Lastly, the code calls s.recv() to read the server’s reply and then prints it.

Note that the string in the `sendall` is modified with a "b" character.  This converts the string into a byte stream. 
 
Edit the code below to include your name in the `sendall` string.  Then execute the cell (SHIFT-RETURN)  to write the file to your current working directory where this jupyter notebook is located.   We will execute the code later in a seperate step.

In [ ]:
%%writefile cs457_echoClient.py

# TCP echo client

import socket

HOST = '127.0.0.1'  # The server's hostname or IP address
PORT = 65432        # The port used by the server

with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.connect((HOST, PORT))
    s.sendall(b'Hello, world')  # modify this to say hello to your name
    data = s.recv(1024)
    
print('Received', repr(data))

#### 2C) Start the server and view the socket state

OK, let's run the server by executing the cell below (SHIFT-RETURN).

When you run the server, it will appear that the jupyter notebook is hung (the `In [*]:` on the left side of the cell doesn't convert into a number indicating the cell has completed execution.)  Don't panic!  The program is simply blocked waiting for input.  

In [ ]:
! python cs457_echoServer.py

To see the current state of sockets on your host, you can use the `netstat` command. It’s available by default on macOS, Linux, and Windows.

Here’s some sample netstat output from macOS after starting the server:

```
$ netstat -an | grep 65432

Active Internet connections (including servers)
Proto Recv-Q Send-Q  Local Address          Foreign Address        (state)  
tcp4       0      0  127.0.0.1.65432        *.*                    LISTEN
```

Start a terminal window and execute the `netstat` command as illustrated above.  You should receive similar output.

We now want to run the client.  Unfortunately, we can't easily execute two separate processes within a single jupyter notebook.  Therefore we will run the client in the separate terminal window you just created.

Within the terminal window, execute the shell command: `python cs457_echoClient.py`. 

You should see the output from the client in the terminal window.  You should also go back to the cell that executed the server.  You will notice that it has stopped, and that it has also printed the IP address and port of the connecting client.  Note that the port number is randomly chosen by the client.



#### 2D)  Running the client/server over a real network

So far you have only run the client and server programs on `localhost` (127.0.0.1).  Now let's run the program across a real network!  

**Step 1:**  Modify the server to listen on IP address 0.0.0.0 (which is a special address that indicates "any available address").  Execute the cell below to save this slightly modified file:  

In [ ]:
%%writefile cs457_echoServer.py

# TCP echo server; performs one echo and then closes and halts

import socket

HOST = '0.0.0.0'    # Listen on available IP address
PORT = 65432        # Port to listen on (non-privileged ports are > 1023)

#use "with" so we don't have to explicitly close the socket
with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:  
    s.bind((HOST, PORT))
    s.listen()
    conn, addr = s.accept()
    with conn:
        print('Connected by', addr)
        while True:
            data = conn.recv(1024)
            if not data:
                break
            conn.sendall(data)

**Step 2:** Next, you will need to get the IP address where the server code is running.  

Execute the `ifconfig` command in the following cell and **note the IP address**.  It will typically be the eth0 field (en0 in Apple MacBooks).  

> If the cell outputs "command not found", edit the cell to execute `/usr/sbin/ifconfig`.
> Also note:  Windows machines don't have ifconfig.  Use ipconfig instead.

In [ ]:
! ifconfig

**Step 3:** Now edit the client code below to use the IP address you just found.  Make sure you also add your name to the echo string. 

Execute the cell to save the program.  

In [ ]:
%%writefile cs457_echoClient.py

# TCP echo client

import socket

HOST = '10.0.2.15'    # MODIFY THIS LINE TO USE THE SERVER'S IP ADDRESS!!!!
PORT = 65432        # The port used by the server

with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.connect((HOST, PORT))
    s.sendall(b'Hello, Jared')  # modify this to say hello to your name
    data = s.recv(1024)
    
print('Received', repr(data))

**Step 4:**  Start the server by executing the cell below.

In [ ]:
! python cs457_echoServer.py

** Step 5:** Final Step.  You need a copy of the echo client on a different computer.  Login to a random linux lab machine.  Use your favorite editor and copy and paste the client code into a file name `remoteEchoClient.py`.  

Now execute this client by running the shell command: `python3 remoteEchoClient.py`.  **Note the use of python version 3!!!  The lab machines default to version 2 which won't work due to the with statement**.


#### What to turn in

After successfully running the echo client, copy and paste the terminal output showing the command and its output (which should contain your name) to a file, then submit this to Canvas.